# 1. Створення бази даних і таблиці за допомогою sqlalchemy

*   **Client** *clients* (id, name, phone)
*   **Order** *orders* (id, client_id, book_id, ordered_date, total, completion_date)
*   **License** *licenses* (id, book_id, expiration_date, title)
*   **Book** *books* (id, translated, printed, isbn)
*   **Worker** *workers* (id, position, name)
*   **Group** *groups* (id, worker_id, book_id)

## init

In [ ]:
# Create database in alchemy

#imported sqllite and sqlalchemy
import sqlalchemy
import sqlite3

# selected path to the database 
from sqlalchemy import create_engine
#engine = create_engine('sqlite:////content/book_store.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)

# created Session
import datetime
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

# created tables in the database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Boolean, Table
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship


Base = declarative_base()
#session = Session()

## Client

In [ ]:
class Client(Base):
  __tablename__ = 'clients'

  id = Column(Integer, primary_key=True, autoincrement=True, unique=True, 
              nullable=False)
  name = Column(String, nullable=False)
  phone = Column(String, nullable=False)

  orders = relationship(
      "Order", 
      backref="client",
      cascade="all, delete")

  def __repr__(self):
    return "<Client(id='%s', name='%s', phone='%s')>"%(self.id, self.name, 
                                                             self.phone)

## Order

In [ ]:
class Order(Base):
  __tablename__ = 'orders'

  id = Column(Integer, primary_key=True, autoincrement=True, unique=True, 
              nullable=False)
  client_id = Column(Integer, ForeignKey('clients.id'), nullable=False)
  book_id = Column(Integer, ForeignKey('books.id'), nullable=False)
  ordered_date = Column(DateTime, nullable=False)
  total = Column(Integer, nullable=False)
  completion_date = Column(DateTime, nullable=True)

  #client = relationship("Client", back_populates="orders")
  #book = relationship("Book", back_populates="orders")

  def __repr__(self):
    ordered_date = self.ordered_date
    completion_date = self.completion_date
    if ordered_date is not None:
      ordered_date = ordered_date.strftime("%d-%b-%Y_%H:%M")
    if completion_date is not None:
      completion_date = completion_date.strftime("%d-%b-%Y_%H:%M")
    return ("<Order(id='{}', client_id='{}', book_id='{}', ordered_date='{}'," +
            "total='{}',completion_date='{}')>").format(
                self.id, self.client_id, self.book_id, 
                ordered_date,
                self.total, completion_date)
    
    # strftime("%d-%b-%Y (%H:%M:%S.%f)")

## Book

In [ ]:
class Book(Base):
  __tablename__ = 'books'

  id = Column(Integer, primary_key=True, autoincrement=True, unique=True, nullable=False)
  translated = Column(Boolean, nullable=True)
  printed = Column(Boolean, nullable=True)
  isbn = Column(String, nullable=False)

  orders = relationship(
      "Order", 
      backref="book",
      cascade="all, delete")
  license = relationship(
      "License", 
      uselist=False, 
      backref="book",
      cascade="all, delete")
  workers = relationship(
    "Worker",
    secondary=lambda: Group,
    backref="books")

  def __repr__(self):
    return "<Book(id='%s', translated='%s', printed='%s', isbn='%s')>" % (
  self.id, self.translated, self.printed, self.isbn)

## License

In [ ]:
class License(Base):
  __tablename__ = 'licenses'

  id = Column(Integer, primary_key=True, autoincrement=True, unique=True, 
              nullable=False)
  book_id = Column(Integer, ForeignKey('books.id'), unique=True, nullable=False)
  expiration_date = Column(DateTime, nullable=False)
  title = Column(String, nullable=False)
  
  #book = relationship( "Book", back_populates="license")

  def __repr__(self):
    expiration_date = self.expiration_date
    if expiration_date is not None:
      expiration_date = expiration_date.strftime("%d-%b-%Y_%H:%M")
    return "<License(id='%s', book_id='%s', expiration_date='%s', title='%s')>" % (
        self.id, self.book_id, expiration_date, self.title)

## Group

In [ ]:
Group = Table('groups', Base.metadata,
    Column('id', Integer, primary_key=True, autoincrement=True, unique=True, 
           nullable=False),
    Column('worker_id', Integer, ForeignKey('workers.id'), nullable=False),
    Column('book_id', Integer, ForeignKey('books.id'), nullable=False)
)


## Worker

In [ ]:
class Worker(Base):
  __tablename__ = 'workers'

  id = Column(Integer, primary_key=True, autoincrement=True, unique=True, 
              nullable=False)
  position = Column(String, nullable=True)
  name = Column(String, nullable=False)

  
  #books = relationship("Book", secondary=association_groups_table, back_populates="workers")

  def __repr__(self):
    return "<Worker(id='%s', position='%s', name='%s')>" % (
        self.id, self.position, self.name)

## create all

In [ ]:
Base.metadata.create_all(engine)

# 2. Dao i DI

## Client

In [ ]:
from abc import ABC, abstractmethod


class AbstractClientDao(ABC):

  @abstractmethod
  def __init__(self, Session):
    pass
    
  @abstractmethod
  def add_client(self, name, phone):
    pass
    
  @abstractmethod
  def get_client(self, id):
    pass
    
  @abstractmethod
  def get_clients(self):
    pass
    
  @abstractmethod
  def update_phone(self, id, phone):
    pass

  @abstractmethod
  def delete(self, id):
    pass


class ClientDao(AbstractClientDao):

  def __init__(self, session):
    self.session = session
    
  def add_client(self, name, phone):
    client = Client(name=name, phone=phone)
    session.add(client)
    return client
    
  def get_client(self, id):
    return session.query(Client).filter_by(id=id).first()
    
  def get_clients(self):
    return session.query(Client).all()
    
  def update_phone(self, id, phone):
    client = session.query(Client).filter_by(id=id).first()
    client.phone = phone
    return client

  def delete(self, id):
    client = session.query(Client).filter(Client.id==id).first()
    if client:
      session.delete(client)


class ClientDao_DI():

  def __init__(self, clientDao):
    self.clientDao = clientDao
    
  def add_client(self, name, phone):
    return self.clientDao.add_client(name, phone)
    
  def get_client(self, id):
    return self.clientDao.get_client(id)
    
  def get_clients(self):
    return self.clientDao.get_clients()
    
  def update_phone(self, id, phone):
    return self.clientDao.update_phone(id, phone)

  def delete(self, id):
    self.clientDao.delete(id)

## Book

In [ ]:
class AbstractBookDao(ABC):

  @abstractmethod
  def __init__(self, session):
    pass
    
  @abstractmethod
  def new_book(self, isbn):
    pass
    
  @abstractmethod
  def add_book(self, isbn):
    pass
    
  @abstractmethod
  def get_book(self, id):
    pass
    
  @abstractmethod
  def get_books(self):
    pass
    
  @abstractmethod
  def update_book(self, id, translated=False, printed=False):
    pass

  @abstractmethod
  def delete(self, id):
    pass
    

class BookDao(AbstractBookDao):

  def __init__(self, session):
    self.session = session
    
  def new_book(self, isbn):    
    book = session.query(Book).filter_by(isbn=isbn).first()
    if book is None:
      book = Book(isbn=isbn)
    return book

  def add_book(self, isbn):
    book = self.new_book(isbn)
    session.add(book)
    return book
    
  def get_book(self, id):
    return session.query(Book).filter_by(id=id).first()
    
  def get_books(self):
    return session.query(Book).all()
    
  def update_book(self, id, translated=False, printed=False):
    book = session.query(Book).filter_by(id=id).first()
    book.translated = translated
    book.printed = printed
    return book

  def delete(self, id):
    book = session.query(Book).filter(Book.id==id).first()
    if book:
      session.delete(book)
  
class BookDao_DI():

  def __init__(self, bookDao):
    self.bookDao = bookDao
    
  def new_book(self, isbn):
    return self.bookDao.new_book(isbn)
    
  def add_book(self, isbn):
    return self.bookDao.add_book(isbn)
    
  def get_book(self, id):
    return self.bookDao.get_book(id)
    
  def get_books(self):
    return self.bookDao.get_books()
    
  def update_book(self, id, translated=False, printed=False):
    return self.bookDao.update_book(id, translated=False, printed=False)

  def delete(self, id):
    self.bookDao.delete(id)

## Order

In [ ]:
# creates orders (and if not present, books) in db


class AbstractOrderDao(ABC):

  @abstractmethod
  def __init__(self, session):
    pass
    
  @abstractmethod
  def add_order(self, client, date, total, book):
    pass
    
  @abstractmethod
  def get_order(self, id):
    pass
    
  @abstractmethod
  def get_orders(self):
    pass
    
  @abstractmethod
  def close_order(self, id, date):
    pass

  @abstractmethod
  def delete(self, id):
    pass
    
    


class OrderDao(AbstractOrderDao):

  def __init__(self, session):
    self.session = session

  def add_order(self, client, date, total, book):
    order = Order(client=client, ordered_date=date, total=total, book=book)
    session.add(order)
    return order
    
  def get_order(self, id):
    return session.query(Order).filter_by(id=id).first()
    
  def get_orders(self):
    return session.query(Order).all()
    
  def close_order(self, id, date):
    order = session.query(Order).filter_by(id=id).first()
    order.completion_date = date
    return order

  def delete(self, id):
    order = session.query(Order).filter(Order.id==id).first()
    if order:
      session.delete(order)


class OrderDao_DI():

  def __init__(self, orderDao):
    self.orderDao = orderDao
    
  def add_order(self, client, date, total, book):
    return self.orderDao.add_order(client, date, total, book)
    
  def get_order(self, id):
    return self.orderDao.get_order(id)
    
  def get_orders(self):
    return self.orderDao.get_orders()
    
  def close_order(self, id, date):
    return self.orderDao.close_order(id, date)

  def delete(self, id):
    self.orderDao.delete(id)

## License

In [ ]:
# creates licenses (if don't exist) in db


class AbstractLicenseDao(ABC):

  @abstractmethod
  def __init__(self, session):
    pass
    
  @abstractmethod
  def new_license(self, book, date, title):
    pass
    
  @abstractmethod
  def add_license(self, book, date, title):
    pass
    
  @abstractmethod
  def get_license(self, id):
    pass
    
  @abstractmethod
  def get_licenses(self):
    pass
    
  @abstractmethod
  def update_title(self, id, title):
    pass

  @abstractmethod
  def delete(self, id):
    pass
    

class LicenseDao(AbstractLicenseDao):

  def __init__(self, session):
    self.session = session
    
  def new_license(self, book, date, title):
    old_license = session.query(License).filter_by(book=book).first()
    if old_license is not None:
      old_license.expiration_date = date
      old_license.title = title
      return old_license
    return License(book=book, expiration_date=date, title=title)

  def add_license(self, book, date, title):
    license = self.new_license(book, date, title)
    session.add(license)
    return license
    
  def get_license(self, id):
    return session.query(License).filter_by(id=id).first()
    
  def get_licenses(self):
    return session.query(License).all()
    
  def update_title(self, id, title):
    license = session.query(License).filter_by(id=id).first()
    license.title = title
    return license

  def delete(self, id):
    license = session.query(License).filter(License.id==id).first()
    if license:
      session.delete(license)
      
class LicenseDao_DI():

  def __init__(self, licenseDao):
    self.licenseDao = licenseDao
    
  def new_license(self, book, date, title):
    return self.licenseDao.new_license(book, date, title)
    
  def add_license(self, book, date, title):
    return self.licenseDao.add_license(book, date, title)
    
  def get_license(self, id):
    return self.licenseDao.get_license(id)
    
  def get_licenses(self):
    return self.licenseDao.get_licenses()
    
  def update_title(self, id, title):
    return self.licenseDao.update_title(id, title)

  def delete(self, id):
    self.licenseDao.delete(id)


## Worker

In [ ]:
class AbstractWorkerDao(ABC):

  @abstractmethod
  def __init__(self, Session):
    pass
    
  @abstractmethod
  def add_worker(self, name, position):
    pass
    
  @abstractmethod
  def get_worker(self, id):
    pass
    
  @abstractmethod
  def get_workers(self):
    pass
    
  @abstractmethod
  def update_position(self, id, position):
    pass

  @abstractmethod
  def delete(self, id):
    pass


class WorkerDao(AbstractWorkerDao):

  def __init__(self, session):
    self.session = session
    
  def add_worker(self, name, position):
    worker = Worker(name=name, position=position)
    session.add(worker)
    return worker
    
  def get_worker(self, id):
    return session.query(Worker).filter_by(id=id).first()
    
  def get_workers(self):
    return session.query(Worker).all()
    
  def update_position(self, id, position):
    worker = session.query(Worker).filter_by(id=id).first()
    worker.position = position
    return worker

  def delete(self, id):
    worker = session.query(Worker).filter(Worker.id==id).first()
    if worker:
      session.delete(worker)
  
class WorkerDao_DI():

  @abstractmethod
  def __init__(self, workerDao):
    self.workerDao = workerDao
    
  @abstractmethod
  def add_worker(self, name, position):
    return self.workerDao.add_worker(name, position)
    
  @abstractmethod
  def get_worker(self, id):
    return self.workerDao.get_worker(id)
    
  @abstractmethod
  def get_workers(self):
    return self.workerDao.get_workers()
    
  @abstractmethod
  def update_position(self, id, position):
    return self.workerDao.update_position(id, position)

  @abstractmethod
  def delete(self, id):
    self.workerDao.delete(id)

## Group

In [ ]:
# creates groups in db (assigns workers to books)
class AbstractGroupDao(ABC):

  @abstractmethod
  def __init__(self, Session):
    pass
    
  @abstractmethod
  def add_group(self, book, worker):
    pass
    
  @abstractmethod
  def get_group(self, id):
    pass
    
  @abstractmethod
  def get_groups(self):
    pass

  @abstractmethod
  def delete(self, id):
    pass


class GroupDao(AbstractGroupDao):

  def __init__(self, session):
    self.session = session
    
  def add_group(self, book, worker):
    book.workers.append(worker)
    return book
    
  def get_group(self, id):
    return session.query(Group).filter_by(id=id).first()
    
  def get_groups(self):
    return session.query(Group).all()

  def delete(self, id):
    group = session.query(Group).filter(Group.c.id==id).first()
    if group:
      worker = session.query(Worker).filter(Worker.id==group.worker_id).first()
      book = session.query(Book).filter(Book.id==group.book_id).first()
      worker.books.remove(book)

class GroupDao_DI():

  def __init__(self, groupDao):
    self.groupDao = groupDao
    
  def add_group(self, book, worker):
    return self.groupDao.add_group(book, worker)
    
  def get_group(self, id):
    return self.groupDao.get_group(id)
    
  def get_groups(self):
    return self.groupDao.get_groups()

  def delete(self, id):
    self.groupDao.delete(id)

## Фабрика

In [ ]:
class DaoFactory():
  clientDao = None
  bookDao = None
  orderDao = None
  licenseDao = None
  workerDao = None
  groupDao = None

  def __new__(cls, session):
    if not hasattr(cls, 'instance'):
      DaoFactory.clientDao = ClientDao_DI(ClientDao(session))
      DaoFactory.bookDao = BookDao_DI(BookDao(session))
      DaoFactory.orderDao = OrderDao_DI(OrderDao(session))
      DaoFactory.licenseDao = LicenseDao_DI(LicenseDao(session))
      DaoFactory.workerDao = WorkerDao_DI(WorkerDao(session))
      DaoFactory.groupDao = GroupDao_DI(GroupDao(session))
      cls.instance = super(DaoFactory, cls).__new__(cls)
    return cls.instance

  def getClientDao(s):
    return DaoFactory.clientDao

  def getBookDao(s):
    return DaoFactory.bookDao

  def getOrderDao(s):
    return DaoFactory.orderDao

  def getLicenseDao(s):
    return DaoFactory.licenseDao

  def getWorkerDao(s):
    return DaoFactory.workerDao

  def getGroupDao(s):
    return DaoFactory.groupDao

# Заповнення бази даних тестовим набором даних

In [ ]:
test_clients_data = [("John", "00007777"),
                     ("Mike", "00014563"),
                     ("Lod", "300566003"),
                     ("Riel", "00007"),
                     ("Alex", "036")]
test_orders_data = [(1, datetime.datetime.now() - datetime.timedelta(weeks=120),
                      775, "775-557-555"),
                     (2, datetime.datetime.now() - datetime.timedelta(weeks=9), 
                      995, "775-333-555"),
                     (3, datetime.datetime.now() - datetime.timedelta(days=2), 
                      775, "999-557-555"),
                     (1, datetime.datetime.now() - datetime.timedelta(hours=2), 
                      1120, "775-557-666"),
                     (4, datetime.datetime.now() - datetime.timedelta(minutes=2), 
                      475, "775-557-555")]
test_licenses_data = [(1, datetime.datetime.now() + datetime.timedelta(weeks=50), 
                      'J.K.Loving "Stone of Potter"'),
                     (2, datetime.datetime.now() + datetime.timedelta(weeks=90), 
                      'Niel Meiman "Ukrainian Gods"'),
                     (3, datetime.datetime.now() + datetime.timedelta(weeks=120), 
                      'Lerry Pratchett "Gort"')]
test_workers_data = [("Johnathan", "Designer"),
                     ("Mickael", "Translator"),
                     ("Wsewolod", "Translator"),
                     ("Gabriel", "Editor"),
                     ("Alexandra", "Editor")]
test_groups_data = [(2, 1),
                    (2, 5),
                    (3, 1),
                    (3, 2),
                    (3, 3),
                    (3, 5)]

def fill_db(session):
  try:
    fact = DaoFactory(session)

    clients = fact.getClientDao()
    for name, phone in test_clients_data:
      clients.add_client(name, phone)
    session.commit()

    books = fact.getBookDao()
    orders = fact.getOrderDao()
    for client_id, order_date, total, isbn in test_orders_data:
      book = books.add_book(isbn)
      orders.add_order(clients.get_client(client_id), order_date, total, book)
    session.commit()

    licenses = fact.getLicenseDao()
    for order_id, expiration_date, title in test_licenses_data:
      licenses.add_license(orders.get_order(order_id).book, expiration_date, title)
    session.commit()

    workers = fact.getWorkerDao()
    for name, position in test_workers_data:
      workers.add_worker(name, position)
    session.commit()

    groups = fact.getGroupDao()
    for order_id, worker_id in test_groups_data:
      groups.add_group(orders.get_order(order_id).book, workers.get_worker(worker_id))
    session.commit()

  except:
    session.rollback()
    raise
  finally:
    session.close()


session = Session()
fill_db(session)

# Друк вмісту бази даних

In [ ]:
def print_clients(clients):
  output = "Clients (n={}):\n".format(len(clients))
  for client in clients:
    output += "~~{}\n".format(client)
  return(output)

def print_books(books):
  output = "Books (n={}):\n".format(len(books))
  for book in books:
    output += "~~{}\n".format(book)
    output += "~~~~ Orders (n={})\n".format(len(book.orders))
    for order in book.orders:
      output += "~~~~~~{}\n".format(order)
    output += "~~~~ Workers (n={})\n".format(len(book.workers))
    for worker in book.workers:
      output += "~~~~~~{}\n".format(worker)
    if book.license is not None:
      output += "~~~~{}\n".format(book.license)
    else:
      output += "~~~~ License (is not present)\n"
  return(output)

def print_orders(orders):
  output = "Orders (n={}):\n".format(len(orders))
  for order in orders:
    output += "~~{}\n".format(order)
    output += "~~~~ {}\n".format(order.client)
    output += "~~~~ {}\n".format(order.book)
  return(output)

def print_licenses(licenses):
  output = "Licenses (n={}):\n".format(len(licenses))
  for license in licenses:
    output += "~~{}\n".format(license)
    output += "~~~~ {}\n".format(license.book)
  return(output)

def print_workers(workers):
  output = "Workers (n={}):\n".format(len(workers))
  for worker in workers:
    output += "~~{}\n".format(worker)
  return(output)

def print_groups(groups):
  output = "Groups (n={}):\n".format(len(groups))
  for group in groups:
    output += "~~<Group(id='{}', book_id='{}', worker_id='{}')>\n".format(
        group.id, group.book_id, group.worker_id)
  return(output)


## print_all(session)

In [ ]:
def print_all(session):
  
  fact = DaoFactory(session)

  clients = fact.getClientDao()
  books = fact.getBookDao()
  orders = fact.getOrderDao()
  licenses = fact.getLicenseDao()
  workers = fact.getWorkerDao()
  groups = fact.getGroupDao()

  all_clients = clients.get_clients()
  all_books = books.get_books()
  all_orders = orders.get_orders()
  all_licenses = licenses.get_licenses()
  all_workers = workers.get_workers()
  all_groups = groups.get_groups()

  print("\n\n((((____----**0**----____))))\n")
  print(print_clients(all_clients))
  print("----**0**----\n")
  print(print_books(all_books))
  print("----**0**----\n")
  print(print_orders(all_orders))
  print("----**0**----\n")
  print(print_licenses(all_licenses))
  print("----**0**----\n")
  print(print_workers(all_workers))
  print("----**0**----\n")
  print(print_groups(all_groups))
  print("----**0**----\n")

print_all(session)

2020-12-05 02:17:52,445 INFO sqlalchemy.engine.base.Engine SELECT clients.id AS clients_id, clients.name AS clients_name, clients.phone AS clients_phone 
FROM clients
2020-12-05 02:17:52,446 INFO sqlalchemy.engine.base.Engine ()
2020-12-05 02:17:52,449 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.translated AS books_translated, books.printed AS books_printed, books.isbn AS books_isbn 
FROM books
2020-12-05 02:17:52,450 INFO sqlalchemy.engine.base.Engine ()
2020-12-05 02:17:52,452 INFO sqlalchemy.engine.base.Engine SELECT orders.id AS orders_id, orders.client_id AS orders_client_id, orders.book_id AS orders_book_id, orders.ordered_date AS orders_ordered_date, orders.total AS orders_total, orders.completion_date AS orders_completion_date 
FROM orders
2020-12-05 02:17:52,454 INFO sqlalchemy.engine.base.Engine ()
2020-12-05 02:17:52,457 INFO sqlalchemy.engine.base.Engine SELECT licenses.id AS licenses_id, licenses.book_id AS licenses_book_id, licenses.expiration_da

# Демонстрація операцій

## вставка

In [ ]:
def test_insert(session):
  try:

    fact = DaoFactory(session)

    clients = fact.getClientDao()
    books = fact.getBookDao()
    orders = fact.getOrderDao()
    licenses = fact.getLicenseDao()
    workers = fact.getWorkerDao()
    groups = fact.getGroupDao()

    client = clients.add_client('New Client', "007-33-55")
    session.commit()
    book = books.add_book('3333-555-4444')
    session.commit()
    orders.add_order(client, datetime.datetime.now(), 100, book)
    session.commit()
    licenses.add_license(book, 
                         datetime.datetime.now() + datetime.timedelta(weeks=130), 
                         'R.M. Oswald "Prankenstein"')
    session.commit()
    worker = workers.add_worker("New Worker", "Editor")
    session.commit()
    groups.add_group(book, worker)
    session.commit()
  except:
    session.rollback()
    raise
  finally:
    session.close()

test_insert(session)

In [ ]:
print_all(session)

## заміна

In [ ]:
def test_update(session):
  try:

    fact = DaoFactory(session)

    clients = fact.getClientDao()
    books = fact.getBookDao()
    orders = fact.getOrderDao()
    licenses = fact.getLicenseDao()
    workers = fact.getWorkerDao()
    groups = fact.getGroupDao()

    clients.update_phone(6, "007-33-55-uuuu")
    session.commit()
    books.update_book(5, translated=True, printed=True)
    session.commit()
    orders.close_order(6, datetime.datetime.now())
    session.commit()
    licenses.update_title(4, 'R.M. Oswald "Prank War"')
    session.commit()
    workers.update_position(6, 'Translator')
    session.commit()
  except:
    session.rollback()
    raise
  finally:
    session.close()

test_update(session)

In [ ]:
print_all(session)

## видалення

In [ ]:
def test_delete(session):
  try:

    fact = DaoFactory(session)

    clients = fact.getClientDao()
    books = fact.getBookDao()
    orders = fact.getOrderDao()
    licenses = fact.getLicenseDao()
    workers = fact.getWorkerDao()
    groups = fact.getGroupDao()

    clients.delete(6)
    session.commit()
    books.delete(1)
    session.commit()
    orders.delete(3)
    session.commit()
    licenses.delete(4)
    session.commit()
    workers.delete(1)
    session.commit()
    groups.delete(7)
    session.commit()
  except:
    session.rollback()
    raise
  finally:
    session.close()

test_delete(session)

In [ ]:
print_all(session)